In [1]:
# Define data
α_1 = 0.1399851
γ_1 = 0.464835
R = 6378*1000. #m 
η_A = 0.29
Q = 1367. #W/m^2
EN = 40
D_r = 5.3 #m
L = 9.777 
k = 1.38064852e-23 #J/K Boltzman constant R/Na
T_s = 135. #K
B = 8.
N = 2000. # pixel width
η = 0.55
λ_v = 500e-9 #nm
f = 2.2e9 #Hz
c = 3e8  #m/s
λ_c = c/f
G_R = η*π*(D_r/λ_c)^2
ρ_A = 10. #kg/m^2
ρ_p = 100. #kg/m^1.5
ρ_T = 2. #kg/m^1.5
ρ_P = 500. #kg*km
ρ_b = 0.001 #kg/J
P_l = 5 #W
μ = 3.986005e14 # m^3/s^2 standard gravitational parameter
X_r = 20. #m/px
m_c = 0.2 #kg, some required weight
η_S = 0.2 # 20% of total mass is structural mass
;

In [2]:
using JuMP
using JuGP
using Ipopt

In [14]:
# Define JuMP model
m = GPModel(solver=IpoptSolver(print_level=0))

@variables m begin
    0 <= D_p <= 10. # payload optical aperture diameter
    0 <= D_T <= 10. # transmitter antenna diameter
    0 <= A <= 10. # solar panel surface area

    # Mass related parameters
    0 <= m_t <= 1000. # total
    0 <= m_b <= 1000. # battery mass
    0 <= m_A <= 1000. # solar panel mass
    0 <= m_p <= 1000. # payload mass
    0 <= m_T <= 1000. # transmitter mass
    0 <= m_S <= 1000. # structural mass
    0 <= m_P <= 1000. # propulsion mass

    # Power related parameters
    0 <= P_t <= 1000. # total power
    0 <= P_T <= 1000. # transmitter power
    0 <= E_b <= 100000. # battery energy

    # Orbit related parameters
    100. <= h <= 40000. # altitude
    R+100. <= a <= 40000.+R # semi-major axis
    5000. <= T <= 100000. # orbit period 
    100. <= r <= 100000. # worst case communications distance
    0 <= d <= 1. # daylight fraction of orbit
    0 <= e <= 1. # eclipse faction of orbit
    0 <= g <= 1. # ground station viewing fraction of orbit
end

# TODO some of these may be better defined as NLexpressions
@NLconstraints m begin
    # Power and communications
    P_t == A*η_A*Q # all power generated by solar panel
    P_T + P_l <= d*P_t
    E_b >= P_t*e*T
    EN*L*k*T_s*R*2*π*N*B*4*r^2 == P_T*G_R*X_r*g*T*η*D_T^2

    # Payload performance
    X_r*D_p == h*λ_v

    # Orbit
#     e >= 1 - d
#     d <= g + 0.5
    a >= R + h
    g == α_1*h^γ_1*R^(-γ_1)
    T == 2*π*a^(3/2)*μ^(-1/2)
    r^2 >= h^2 + 2*R*h 

    # Mass budgets
    m_b == ρ_b*E_b
    m_p == ρ_p*D_p^1.5
    m_A == ρ_A*A
    m_T == ρ_T*D_T^1.5
    m_P == ρ_P/h
    m_S == η_S*m_t
    m_t >= m_b + m_p + m_A + m_T + m_P + m_S
end

@NLobjective(m, Min, m_t)
;

In [16]:
solve(m)

:Infeasible